# dim_geolocalizacao

Esse notebook é responsável por criar uma tabela com as informações básicas das localizações.


## Configurações Iniciais

### Importando as Bibliotecas

In [0]:
from pyspark.sql import functions as F

### Carregando os Dados

In [0]:
df_geolocation_dataset = (
    spark.read.csv(
        "/Volumes/workspace/default/e_commerce/olist_geolocation_dataset.csv",
        header=True,
        inferSchema=True
    )
)

## Transformação dos Dados

In [0]:
df_final_geolocation_dataset = (
    df_geolocation_dataset
    .withColumn(
        "geolocation_country",
        F.lit("BR")
    )
    .withColumn(
        "geolocation_city",
        F.upper(F.col("geolocation_city"))
    )
    .withColumn(
        "geolocation_city_state",
        F.concat_ws(
            " - ",
            F.col("geolocation_city"),
            F.col("geolocation_state")
        )
    )
)

## Salvamento dos Dados

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ifrs_dev_ecommerce.gold.dim_geolocalizacao (
    geolocation_zip_code_prefix INT PRIMARY KEY COMMENT "Prefixo do CEP da geolocalização",
    geolocation_city STRING COMMENT "Cidade da geolocalização",
    geolocation_state STRING COMMENT "Estado da geolocalização",
    geolocation_country STRING COMMENT "País da geolocalização",
    geolocation_city_state STRING COMMENT "Cidade e estado da geolocalização"
)
USING DELTA
CLUSTER BY (geolocation_zip_code_prefix, geolocation_city_state)
TBLPROPERTIES ("quality"="gold", "layer"="data-engineering")
COMMENT "Dimensão de geolocalização contendo informações de localização.";

-- Adicionando tags à tabela
ALTER TABLE ifrs_dev_ecommerce.gold.dim_geolocalizacao SET TAGS ("quality"="gold", "layer"="data-engineering");

In [0]:
(
    df_final_geolocation_dataset
    .write.mode("overwrite")
    .format("delta")
    .saveAsTable("ifrs_dev_ecommerce.gold.dim_geolocalizacao")
)